# Impact Analysis of GoodThought NGO Initiative

![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


# Preview each table

In [13]:
-- Preview the assignments table
SELECT *
FROM
	public.assignments
LIMIT 5;

,assignment_id,assignment_name,start_date,end_date,budget,region,impact_score
0,1,Assignment_1,2021-10-17,2021-12-04,-32322.03,West,5.55
1,2,Assignment_2,2020-10-26,2020-11-28,57278.40,South,1.45
2,3,Assignment_3,2021-08-11,2022-03-17,40414.51,West,2.34
3,4,Assignment_4,2021-11-22,2022-05-17,31732.48,East,7.05
4,5,Assignment_5,2020-11-22,2021-07-10,13548.22,North,5.29


In [14]:
-- Preview the donations table
SELECT *
FROM
	public.donations
LIMIT 5;

,donation_id,donor_id,amount,donation_date,assignment_id
0,1,2733,271.36,2021-08-21 00:00:00+00:00,4226
1,2,2608,251.49,2021-10-15 00:00:00+00:00,1323
2,3,1654,528.38,2020-03-03 00:00:00+00:00,4881
3,4,3265,730.36,2021-02-06 00:00:00+00:00,1237
4,5,4932,285.96,2022-03-05 00:00:00+00:00,1626


In [15]:
--Preview the donors table
SELECT *
FROM 
	public.donors
LIMIT 5;

,donor_id,donor_name,donor_type
0,1,Donor_1,Individual
1,2,Donor_2,Organization
2,3,Donor_3,Individual
3,4,Donor_4,Organization
4,5,Donor_5,Organization


## Question 1
List the top five assignments based on total value of donations, categorized by donor type. The output should include four columns: 1) assignment_name, 2) region, 3) rounded_total_donation_amount rounded to two decimal places, and 4) donor_type, sorted by rounded_total_donation_amount in descending order. Save the result as highest_donation_assignments.

In [10]:
-- highest_donation_assignments

-- Use a CTE to retrieve total donation ammounts
-- for each assignment, grouped by donor type
WITH donation_details AS (
	SELECT 
		d.assignment_id, 
		donors.donor_type, 
		ROUND(SUM(d.amount),2) AS rounded_total_donation_amount
	FROM 
		public.donations AS d
	JOIN public.donors AS donors
		ON d.donor_id = donors.donor_id
	GROUP BY 
		d.assignment_id, donors.donor_type
)
-- Join the CTE with the assignment table to retrieve region labels
SELECT 
	a.assignment_name, 
	a.region,
	donation_details.rounded_total_donation_amount,
	donation_details.donor_type
FROM public.assignments AS a
JOIN donation_details
	ON a.assignment_id = donation_details.assignment_id
-- Order results by total donation amount in descending order
ORDER BY rounded_total_donation_amount DESC
LIMIT 5;


--to save

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


## Question 2
Identify the assignment with the highest impact score in each region, ensuring that each listed assignment has received at least one donation. The output should include four columns: 1) assignment_name, 2) region, 3) impact_score, and 4) num_total_donations, sorted by region in ascending order. Include only the highest-scoring assignment per region, avoiding duplicates within the same region. Save the result as top_regional_impact_assignments.

In [11]:
-- top_regional_impact_assignments

-- Get number of donations each assignment received
WITH number_of_donations AS(
	SELECT 
		d.assignment_id,
		COUNT(d.donation_id) AS num_total_donations
	FROM 
		public.donations AS d
	GROUP BY 
		d.assignment_id
),
-- Rank each assignment within its region by impact score
a_rank AS (
	SELECT
		a.assignment_name,
		a.region,
		a.impact_score,
		nd.num_total_donations,
	-- Use window function to rank assignments within each region
		ROW_NUMBER() OVER(PARTITION BY a.region ORDER BY a.impact_score DESC) AS region_rank
	FROM 
		public.assignments AS a
	JOIN number_of_donations AS nd
		ON a.assignment_id = nd.assignment_id
	-- Return only assignments that recieved at least one donation
	WHERE 
		nd.num_total_donations > 0
)
-- Extract the highest ranking assignments within each region
SELECT
	assignment_name,
	region,
	impact_score,
	num_total_donations
FROM 
	a_rank
WHERE 
	region_rank = 1
ORDER BY 
	region;


,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2
